# Data Mining Course Spark Exercise
## Sharif University of Technology

In this notebook we are going to analyze farsi wikipedia. 
Outline of the exercise:
* Dataset preparation (5 Points)
* Preprocessing (35 Points) 
* Exploration (20 Points) 
* TF-IDF + Search (40 Points)

You should replace the `TODO` parts with your implementation. Remeber that each `TODO` may take multiple lines and you shouldn't limit your self to one-line codes.

## Prerequisites
You should be faimilar with [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). In this notebook you should use the following formula for tf-idf:
$$f_{t,d}/len(d) \times log(1 + \frac{N}{n_t})$$

## Warning: RDD api only
You **can not** use Dataframe, Dataset, mllib, ml, ... apis of spark in this exercise. You should only use the [RDD api](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD).

# Section 0: Please enter your name below
# Name: امین کشیری
# Student Number: 97101026

# Section 1: Dataset preparation (5 Points)

This section of notebook contains only shell commands. You don't need to completely understand each command or change anything.

Please run all the paragraphs sequentially.

In [ ]:
! sudo apt-get -y install wget git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.9).
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


## Download the dump

In [ ]:
! wget --no-check-certificate -P . https://dumps.wikimedia.your.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2

--2021-11-06 19:24:01--  https://dumps.wikimedia.your.org/fawiki/latest/fawiki-latest-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.your.org (dumps.wikimedia.your.org)... 204.9.55.82, 2001:4978:1:420::cc09:3752
Connecting to dumps.wikimedia.your.org (dumps.wikimedia.your.org)|204.9.55.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002962062 (956M) [application/x-bzip]
Saving to: ‘./fawiki-latest-pages-articles-multistream.xml.bz2’

fawiki-latest-pages 100%[===================>] 956.50M   154MB/s    in 5.9s    

2021-11-06 19:24:07 (161 MB/s) - ‘./fawiki-latest-pages-articles-multistream.xml.bz2’ saved [1002962062/1002962062]



## Extract the dump (this may take a few minutes)

In [ ]:
! bzip2 -d fawiki-latest-pages-articles-multistream.xml.bz2

## Clone git project for converting wikipedia xml dump to json

In [ ]:
! pip install wikiextractor

     |████████████████████████████████| 46 kB 3.0 MB/s 


## Run the script to convert xml to json (this might take around 30 minutes)

In [ ]:
! python -m wikiextractor.WikiExtractor --json fawiki-latest-pages-articles-multistream.xml

INFO: Preprocessing 'fawiki-latest-pages-articles-multistream.xml' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Preprocessed 1600000 pages
INFO: Preprocessed 1700000 pages
INFO: Preprocessed 1800000 pages
INFO: Preprocessed 1900000 pages
INFO: Preprocessed 2000000 pages
INFO: Preprocessed 2100000 pages
INFO: Preprocessed 2200000 pages
INFO: Preprocessed 2300000 pages
INFO: Preprocessed 2400000 pages
INFO: Preprocessed 2500000 pages
INFO: Preprocessed 2600000 pages
INFO: Preprocessed 2700000

## Ensure output files exist

In [ ]:
! ls text/*/*

## Save extracted files

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# cp -r text drive/MyDrive/Colab\ Notebooks/MDA/HW2

## Copy from drive instead of extracting again

In [ ]:
# !mkdir text
# !cp -r drive/MyDrive/Colab\ Notebooks/MDA/HW2/text/* text
# !du -hs * #check if 3.0G data is there

## Install Pypark & Initialization
Uncomment this section if you use google colab or local pc

In [ ]:
! pip install pyspark

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 59.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=6eabf187df2da8d109366cf6c092cf84e69afc847c762ac8c23a6128b9f85e10
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .config('spark.ui.port', '4050') \
    .appName("MDA_2021") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

#### Run Spark UI for colab

In [ ]:
# !wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip -n -q ngrok-stable-linux-amd64.zip

In [ ]:
# get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
# !curl -s http://localhost:4040/api/tunnels 

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://afd3-35-222-127-41.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://afd3-35-222-127-41.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


## Reading the data

In [ ]:
LOAD = True
path = 'text/*/*'
if LOAD:
  from google.colab import drive
  drive.mount('/content/drive')
  path = 'drive/MyDrive/Colab\ Notebooks/MDA/HW2/text/*/*'
articles_rdd_orig = sc.textFile(path) # Now you have a RDD with wikipedia posts
articles_rdd_orig.take(2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['{"id": "2", "revid": "23528", "url": "https://fa.wikipedia.org/wiki?curid=2", "title": "\\u0635\\u0641\\u062d\\u0647\\u0654 \\u0627\\u0635\\u0644\\u06cc", "text": "&lt;templatestyles src=\\"\\u0635\\u0641\\u062d\\u0647 \\u0627\\u0635\\u0644\\u06cc/styles.css\\"/&gt;\\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n \\n\\u0627\\u0645\\u0631\\u0648\\u0632: \\u060c \\u0645\\u06cc\\u0644\\u0627\\u062f\\u06cc \\u0628\\u0631\\u0627\\u0628\\u0631 \\u0647\\u062c\\u0631\\u06cc \\u062e\\u0648\\u0631\\u0634\\u06cc\\u062f\\u06cc \\u0648 (UTC)\\n \\n \\n \\n \\n \' \\u2013 \'\\n \\n \\n \\n \\n \\n \\n __NOEDITSECTION__"}',
 '{"id": "594", "revid": "987083", "url": "https://fa.wikipedia.org/wiki?curid=594", "title": "\\u0648\\u06cc\\u06a9\\u06cc\\u200c\\u067e\\u062f\\u06cc\\u0627", "text": "\\u0648\\u06cc\\u06a9\\u06cc\\u200c\\u067e\\u062f\\u06cc\\u0627 (\\u06a9\\u0648\\u062a\\u0647\\u200c\\u0646\\u0648\\u0634\\u062a \\u0628\\u0647\\u200c\\u0635\\u0648\\u0631\\u062a \\u00ab\\u0648\\u067e\\u00bb \\u0

In [ ]:
cat text/AA/wiki_00

# Section 2: Preprocessing (35 Points)

In this section we will remove useless part (for example /n and /u200c) also find and remove stop words and remove the words with a low count in the corpus

In [ ]:
# env variables

SAMPLE = True
SAMPLE_SIZE = 0.001

In [ ]:
if SAMPLE:
  articles_rdd = articles_rdd_orig.sample(True, SAMPLE_SIZE)
  articles_rdd = articles_rdd.coalesce(10)
  articles_rdd.cache()

articles_rdd.take(2)

['{"id": "2977", "revid": "33049885", "url": "https://fa.wikipedia.org/wiki?curid=2977", "title": "\\u0639\\u0644\\u0645 (\\u0639\\u0632\\u0627\\u062f\\u0627\\u0631\\u06cc)", "text": "\\u0639\\u064e\\u0644\\u0645 \\u06cc\\u0627 \\u062c\\u0631\\u06cc\\u062f\\u0647 \\u06cc\\u0627 \\u062a\\u0648\\u063a \\u06cc\\u06a9\\u06cc \\u0627\\u0632 \\u0648\\u0633\\u06cc\\u0644\\u0647\\u200c\\u0647\\u0627\\u06cc \\u0646\\u0645\\u0627\\u062f\\u06cc\\u0646 \\u0628\\u06a9\\u0627\\u0631 \\u0631\\u0641\\u062a\\u0647 \\u062f\\u0631 \\u0622\\u06cc\\u06cc\\u0646 \\u0633\\u0648\\u06af\\u0648\\u0627\\u0631\\u06cc \\u0639\\u0627\\u0634\\u0648\\u0631\\u0627 \\u062f\\u0631 \\u0645\\u06cc\\u0627\\u0646 \\u0634\\u06cc\\u0639\\u06cc\\u0627\\u0646 \\u0627\\u0633\\u062a. \\u0627\\u06cc\\u0646 \\u0648\\u0633\\u06cc\\u0644\\u0647 \\u0645\\u0639\\u0645\\u0648\\u0644\\u0627\\u064b \\u0686\\u0648\\u0628\\u06cc \\u0628\\u0644\\u0646\\u062f\\u06cc \\u0628\\u0647 \\u0627\\u0631\\u062a\\u0641\\u0627\\u0639 \\u067e\\u0646\\u06

In [ ]:
import json

#TODO: parse the json string

articles_rdd_json = articles_rdd.map(lambda x: json.loads(x))
articles_rdd_json.take(2)

In [ ]:
# BAD_WORDS = ['\u200c', '\n', '.', ',' , '(', ')', '!', '~', '?', ';', '،', '[', ']', '"', "'", '«', '»', ':', '؛']
# def remove_bad_words(x):
#   for bad_word in BAD_WORDS:
#     x['text'] = x['text'].replace(bad_word, ' ')
#     x['title'] = x['title'].replace(bad_word, ' ')
#   return x

import re
def remove_bad_words(x):
  x['text'] = re.sub(r'\W+', ' ', x['text'])
  x['title'] = re.sub(r'\W+', ' ', x['title'])
  return x

cleansed_articles_rdd = articles_rdd_json.map(remove_bad_words)
cleansed_articles_rdd.take(2)

In [ ]:
words_rdd = cleansed_articles_rdd.flatMap(lambda x: x['title'].split() + x['text'].split())
# words_rdd.cache().count()
words_rdd.take(5)

['استان', 'مازندران', 'استان', 'ماز', 'ند']

In [ ]:
words_count_rdd = words_rdd.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
words_count_rdd.take(5)

[('و', 5119), ('دریای', 63), ('می', 2297), ('گلستان', 8), ('تهران', 54)]

In [ ]:
top_100 = sc.parallelize(words_count_rdd.sortBy(lambda x: -x[1]).take(100))
top_100.collect()[:10]

[('در', 5715),
 ('و', 5119),
 ('به', 3967),
 ('از', 3379),
 ('است', 2744),
 ('که', 2454),
 ('می', 2297),
 ('این', 1779),
 ('را', 1726),
 ('های', 1521)]

In [ ]:
stopwords = top_100.map(lambda x: x[0]).collect()
# stopwords
stopwords = [
'در',
 'و',
 'به',
 'از',
 'است',
 'که',
 'می',
 'این',
 'را',
 'های',
 'با',
 'یک',
 'شده',
 'آن',
 'ها',
 'شد',
 'بود',
 'برای',
 'کرد',
 'او',
 'شود',
 'ای',
 'دارد',
 'بر',
 'خود',
 'تا',
 'یا',
 'وی',
 'کند',
 'نیز',
 'باشد',
 'پس',
 'بوده',
 'یکی',
 'کرده',
 'هم',
 'توسط',
 'بین',
 'دیگر',
 'اس',
 'همچنین',
 'اند',
 'اما',
 'کنند',
 'هر',
 'هایی',
 'پیش',
 'داشته',
 'داد',
 'روی',
 'داشت',
 'هستند',
 'بودند',
 'کردند',
 'داده',
 'زیر',   
]
print(stopwords)

['در', 'و', 'به', 'از', 'است', 'که', 'می', 'این', 'را', 'های', 'با', 'یک', 'شده', 'آن', 'ها', 'شد', 'بود', 'برای', 'کرد', 'او', 'شود', 'ای', 'دارد', 'بر', 'خود', 'تا', 'یا', 'وی', 'کند', 'نیز', 'باشد', 'پس', 'بوده', 'یکی', 'کرده', 'هم', 'توسط', 'بین', 'دیگر', 'اس', 'همچنین', 'اند', 'اما', 'کنند', 'هر', 'هایی', 'پیش', 'داشته', 'داد', 'روی', 'داشت', 'هستند', 'بودند', 'کردند', 'داده', 'زیر']


In [ ]:
# def f(x):
#   for stopword in stopwords:
#     x['text'] = re.sub(rf'\b{stopword}\b', ' ', x['text'])
#     x['title'] = re.sub(rf'\b{stopword}\b', ' ', x['title'])
#   return x

def remove_stopwords(x):
  for stop_word in stopwords:
    with_space = ' ' + stop_word + ' '
  
    x['text'] = x['text'].replace(with_space, ' ')
  
    if x['text'].startswith(stop_word + ' '):
      x['text'] = x['text'].replace(stop_word, ' ', 1)

    if x['text'].endswith(' ' + stop_word):
      x['text'] = ' '.join(x['text'].rsplit(stop_word, 1))



    x['title'] = x['title'].replace(with_space, ' ')
  
    if x['title'].startswith(stop_word + ' '):
      x['title'] = x['title'].replace(stop_word, ' ', 1)

    if x['title'].endswith(' ' + stop_word):
      x['title'] = ' '.join(x['title'].rsplit(stop_word, 1))
  
  return x


articles_without_stopwords_rdd = cleansed_articles_rdd.map(remove_stopwords)
# articles_without_stopwords_rdd.cache()
articles_without_stopwords_rdd.take(2)

[{'id': '1679',
  'revid': '740480',
  'text': ' ان ماز ند ران مرکزیت شهر ساری شمال ایران کرانه جنوبی دریای مازندران مرز استان گلستان سمنان تهران البرز قزوین گیلان استان پرجمعیت ترین مناطق ایران لحاظ تراکم جمعیتی دارای ۲۲ شهرستان مازندران پر افتخارترین استان ایران بازی المپیک قله دماوند مرتفع ترین کوه ایران بلندترین آتشفشان آسیا خاورمیانه مازندران شهرستان آمل قرار مازندران وسعتی معادل ۱ ۴۶ درصد ایران برخوردار قرار گرفتن ساحل جنوبی بزرگترین دریاچه جهان نام دریای مازندران خزر همجواری چهار کشور ساحلی دریا یعنی ترکمنستان قزاقستان روسیه جمهوری آذربایجان سو واقع شدن شمال کلان شهر تهران سوی امتیاز حمل نقل دریایی کشورهای همسایه طریق منطقه ویژه اقتصادی بنادر امیرآباد فریدون کنار بندر نوشهر اسکله سکوی نفتی نکا شرق مازندران اتصال راه آهن سراسری ایران وجود فرودگاه المللی ساری نوشهر رامسر برخورداری سه جاده شوسه ارتباطی استان تهران اقدام سرمایه گذاری طرح دست ساخت آزادراه تهران شمال امکان دسترسی سریع فاصله کوتاه ۱۲۱ کیلومتر ارتباط تهران ـ مازندران عبور خطوط المللی فیبر نوری شبکه ارتباطی مسیر کریدور ا

In [ ]:
MIN_COUNT = 20

uncommon_words = words_count_rdd.filter(lambda x: x[1]<MIN_COUNT)
# print(uncommon_words.count())
# uncommon_words.take(10)

uncommon_words = uncommon_words.map(lambda x: x[0])
uncommon_words = uncommon_words.collect()
print(len(uncommon_words))
# uncommon_words


# commmon_words = ~uncommon

17348


In [ ]:
uncommon_words_100 = uncommon_words[:100]
def remove_uncommon_words(x):
  for uncommon_word in uncommon_words_100:
    with_space = ' ' + uncommon_word + ' '
  
    x['text'] = x['text'].replace(with_space, ' ')
  
    if x['text'].startswith(uncommon_word + ' '):
      x['text'] = x['text'].replace(uncommon_word, ' ', 1)

    if x['text'].endswith(' ' + uncommon_word):
      x['text'] = ' '.join(x['text'].rsplit(uncommon_word, 1))



    x['title'] = x['title'].replace(with_space, ' ')
  
    if x['title'].startswith(uncommon_word + ' '):
      x['title'] = x['title'].replace(uncommon_word, ' ', 1)

    if x['title'].endswith(' ' + uncommon_word):
      x['title'] = ' '.join(x['title'].rsplit(uncommon_word, 1))
  
  return x


articles_cleaned_rdd = articles_without_stopwords_rdd.map(remove_uncommon_words)
# articles_cleaned_rdd.cache()
articles_cleaned_rdd.take(5)

In [ ]:
import json
import re

def remove_bad_words(x):
  x['text'] = re.sub(r'\W+', ' ', x['text'])
  x['title'] = re.sub(r'\W+', ' ', x['title'])
  return x

def remove_stopwords(x):
  for stop_word in stopwords:
    with_space = ' ' + stop_word + ' '
  
    x['text'] = x['text'].replace(with_space, ' ')
  
    if x['text'].startswith(stop_word + ' '):
      x['text'] = x['text'].replace(stop_word, ' ', 1)

    if x['text'].endswith(' ' + stop_word):
      x['text'] = ' '.join(x['text'].rsplit(stop_word, 1))



    x['title'] = x['title'].replace(with_space, ' ')
  
    if x['title'].startswith(stop_word + ' '):
      x['title'] = x['title'].replace(stop_word, ' ', 1)

    if x['title'].endswith(' ' + stop_word):
      x['title'] = ' '.join(x['title'].rsplit(stop_word, 1))
  
  return x

def remove_uncommon_words(x):
  for uncommon_word in uncommon_words_100:
    with_space = ' ' + uncommon_word + ' '
  
    x['text'] = x['text'].replace(with_space, ' ')
  
    if x['text'].startswith(uncommon_word + ' '):
      x['text'] = x['text'].replace(uncommon_word, ' ', 1)

    if x['text'].endswith(' ' + uncommon_word):
      x['text'] = ' '.join(x['text'].rsplit(uncommon_word, 1))



    x['title'] = x['title'].replace(with_space, ' ')
  
    if x['title'].startswith(uncommon_word + ' '):
      x['title'] = x['title'].replace(uncommon_word, ' ', 1)

    if x['title'].endswith(' ' + uncommon_word):
      x['title'] = ' '.join(x['title'].rsplit(uncommon_word, 1))
  
  return x


# env variables
SAMPLE = True
SAMPLE_SIZE = 0.01

# articles_rdd = #TODO: parse the json string
if SAMPLE:
  articles_rdd = articles_rdd_orig.sample(True, SAMPLE_SIZE)
  articles_rdd = articles_rdd.coalesce(10)
  articles_rdd.cache()
else:
  articles_rdd = articles_rdd_orig

articles_rdd_json = articles_rdd.map(lambda x: json.loads(x))


# cleansed_articles_rdd= #TODO: cleansed text
cleansed_articles_rdd = articles_rdd_json.map(remove_bad_words)


# words_rdd = #TODO: extract words from title and description
words_rdd = cleansed_articles_rdd.flatMap(lambda x: x['title'].split() + x['text'].split())


# words_count_rdd = #TODO: make an rdd with the count of each word
words_count_rdd = words_rdd.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)

# top_100 = #TODO: find the 100 most common words
top_100 = sc.parallelize(words_count_rdd.sortBy(lambda x: -x[1]).take(100))

# stopwords = ['و', 'با'] #TODO: complete the list of stopwords based on top 100 common words
stopwords = ['در', 'و', 'به', 'از', 'است', 'که', 'می', 'این', 'را', 'های', 'با', 'یک', 'شده', 'آن', 'ها', 'شد', 'بود', 'برای', 'کرد', 'او', 'شود', 'ای', 'دارد', 'بر', 'خود', 'تا', 'یا', 'وی', 'کند', 'نیز', 'باشد', 'پس', 'بوده', 'یکی', 'کرده', 'هم', 'توسط', 'بین', 'دیگر', 'اس', 'همچنین', 'اند', 'اما', 'کنند', 'هر', 'هایی', 'پیش', 'داشته', 'داد', 'روی', 'داشت', 'هستند', 'بودند', 'کردند', 'داده', 'زیر']


# articles_without_stopwords_rdd = #TODO: remove stopwords from the article title and text
articles_without_stopwords_rdd = cleansed_articles_rdd.map(remove_stopwords)


MIN_COUNT = 20

# uncommon_words = #TODO: list of the words that have occured less than MIN_COUNT in the whole corpus
uncommon_words = words_count_rdd.filter(lambda x: x[1]<MIN_COUNT)
uncommon_words = uncommon_words.map(lambda x: x[0])
uncommon_words = uncommon_words.collect()


# articles_cleaned_rdd = #TODO: remove uncommon words from articles_without_stopwords_rdd
uncommon_words_100 = uncommon_words[:100]
articles_cleaned_rdd = articles_without_stopwords_rdd.map(remove_uncommon_words)

articles_cleaned_rdd.take(1) # This should output a dictionary with url,title and text keys. title and text should not have stopwords or uncommon.

[{'id': '637',
  'revid': '728972',
  'text': 'محتوای آزاد اطلاعات آزاد کار عملی هنری کارهای خلاقانه مطابق تعریف آثار فرهنگی آزاد محدودیت قانونی چشمگیری آزادی افراد کاربرد توزیع تغییر توزیع کارهای منتج محتوا نداشته محتوای آزاد محتوای باز متمایز محتوای آزاد توان دست برد حالی اگر محتوایی باز آزاد نباشد ممکن نتوان دست برد جا واژه محتوا طیف گسترده موسیقی کتاب نرم افزار چندرسانه دربرمی گیرد محتوای آزاد آزادی اشاره محتوای آزاد همه کارهای مالکیت عمومی کارهای دارای کپی رایت اجازه نامه شان آزادی برشمرده بالا محترم بدانند دربرمی گیرد جا بیش تر کشورها قانون کپی رایت اجازه کنترل انحصاری اثر دهد محتواهایی مالکیت عمومی نیستند آزاد شوند باید صریحا معمولا درون اثر اعلام محتوا دارای اجازه نامه آزاد آزاد استفاده منظور اهداف گوناگون تجاری کس تواند نسخه برداری تغییر دهد توزیع اجازه نامه مستندات آزاد گنو کریتیو کامانز اجازه نامه بی دی نمونه اجازه نامه محتوای آزاد ویکی پدیا تحت اجازه نامه کرییتیو کامنز نوع ارجاع اشتراک همسان CC Attribution Share Alike منتشر تضمین کننده آزادی همیشگی تعریف دانش آزاد دانش باز 

# Section 3: Exploration (20 Points)
Please answer the following questions regarding the dataset:
* How many unique words remain after the cleaning procedure?
* What are the top 100 most common trigrams in the corpus?
* Plot a distribution from document lengths using appropriate bin sizes with 100 bins
* What are the top 100 most common english words in the corpus?
* What is the url of the longest article?
* How many articles contain your first name?!

#### How many unique words remain after the cleaning procedure?

In [ ]:
words_count_after_cleaning = words_count_rdd.filter(lambda x: x[1]>=MIN_COUNT)
# words_after_cleaning = words_after_cleaning.map(lambda x: x[0])
words_count_after_cleaning.count()
# words_count_after_cleaning.take(10)

673

### What are the top 100 most common trigrams in the corpus?

In [ ]:
trigrams = words_count_after_cleaning.filter(lambda x: len(x[0])==3).sortBy(lambda x: x[1], ascending=False).take(100)
trigrams[:5]

[('است', 2402), ('این', 1239), ('های', 1084), ('شده', 737), ('سال', 690)]

### Plot a distribution from document lengths using appropriate bin sizes with 100 bins

In [ ]:
import matplotlib.pyplot as plt
import math
doc_lengths = articles_cleaned_rdd.map(lambda x: len(x['text']))
doc_lengths = doc_lengths.filter(lambda x: x != 0)

sd = math.sqrt(doc_lengths.variance())
mean = doc_lengths.mean()

print('sd is: ', sd)
print('mean is: ', mean)

doc_lenghts_outliers_removed = doc_lengths.filter(lambda x: mean -3*sd <x<mean + 3*sd)


hist = doc_lenghts_outliers_removed.histogram(100)


plt.hist(hist[0][:-1] , hist[0], weights=hist[1])
plt.show()

# print(hist[1])
# print(hist[0])
# print(sd)
# print(mean)


sd is:  1258.1372240088263
mean is:  391.2329873125721


tuple

### What are the top 100 most common english words in the corpus?

In [ ]:
import re

pattern = re.compile(r"^[A-Za-z]+$")

top100_english = words_count_after_cleaning.filter(lambda x: not pattern.match(x[0]) is None).sortBy(lambda x: x[1], ascending=False)
top100_english.take(100)


[]

### What is the url of the longest article?

In [ ]:
urls_lengths = articles_cleaned_rdd.map(lambda x: (len(x['text']), x['url'])).sortBy(lambda x: x[0], ascending=False)
urls_lengths.take(1)

[(28340, 'https://fa.wikipedia.org/wiki?curid=88953'),
 (10052, 'https://fa.wikipedia.org/wiki?curid=294603'),
 (7943, 'https://fa.wikipedia.org/wiki?curid=5105595'),
 (6748, 'https://fa.wikipedia.org/wiki?curid=5055829'),
 (5954, 'https://fa.wikipedia.org/wiki?curid=34881'),
 (5469, 'https://fa.wikipedia.org/wiki?curid=4580799'),
 (5168, 'https://fa.wikipedia.org/wiki?curid=2066291'),
 (5144, 'https://fa.wikipedia.org/wiki?curid=208055'),
 (5027, 'https://fa.wikipedia.org/wiki?curid=115433'),
 (4980, 'https://fa.wikipedia.org/wiki?curid=387759')]

### How many articles contain your first name?!

In [ ]:
name = 'امین'
articles_cleaned_rdd.filter(lambda x: name in x['text']).count()

9

# Section 4: TF-IDF + Search (40 Points)

In [ ]:
word_df_rdd = #TODO: calculate document frequncy for each word
articles_tf_idf_vectors = #TODO: add `vector` key to articles_cleaned_rdd dictionary with the tf_idf dictionary

In [ ]:
import math
num_articles = articles_cleaned_rdd.count()


word_df_rdd = articles_cleaned_rdd.flatMap(lambda x: [(word, 1)  for word in set(x['text'].split() + x['title'].split()) ] ).reduceByKey(lambda a,b: a+b)
word_idf_rdd = word_df_rdd.map(lambda x: (x[0], math.log(num_articles/x[1])) )

word_tf_rdd = articles_cleaned_rdd.flatMap(lambda x: [((word, x['id']), 1) for word in x['title'].split() + x['text'].split()] ).reduceByKey(lambda a,b: a+b).map(lambda x: (x[0][0], (x[0][1], x[1])))


articles_tf_idf_vectors = word_tf_rdd.join(word_idf_rdd)
articles_tf_idf_vectors = articles_tf_idf_vectors.map(lambda x: (x[1][0][0], (x[0], x[1][0][1]*x[1][1]) ) )
articles_tf_idf_vectors = articles_tf_idf_vectors.groupByKey().mapValues(dict)

articles_cleaned_rdd_with_vector = articles_cleaned_rdd.map(lambda x: (x['id'], x)).join(articles_tf_idf_vectors).map(lambda x: {**x[1][0], 'vector':x[1][1]})
articles_cleaned_rdd_with_vector.take(1)



[{'id': '21831',
  'revid': '586129',
  'text': 'بابک احمدی زاده ۱۳۲۷ تهران نویسنده مؤلف مترجم منتقد هنری پژوهشگر ایرانی آثار متون پرفروش بازار کتاب فلسفی ایران بیشتر نشر مرکز چاپ زندگی سال ۱۳۲۷ تهران دنیا آمده مقیم همین شهر احمدی سال ۱۳۴۹ مدرک کارشناسی علوم سیاسی دانشگاه تهران دریافت سپس ادامه تحصیل پنسیلوانیای آمریکا رفت مقطع کارشناسی ارشد فلسفه دانشگاه پنسیلوانیا فارغ التحصیل احمدی سالهای ۱۳۵۳ ۱۳۵۴ لندن سال بعد بهمن ۱۳۵۷ پاریس گذرانده فعالیت سیاسی فعالیت سیاسی علیه حکومت محمدرضا پهلوی چنین سال اخیر منتقدان سیاست جمهوری اسلامی انتخابات ریاست جمهوری سال ۱۳۸۸ همراه خشایار دیهیمی انتشار بیانیه دفاع اصلاح طلبی انتخابات پرداختند تأکید کارنامه چهارساله آقای احمدی نژاد اسباب سرشکستگی حکومت تحقیر ملی نقدها بیشتر آثار نخستین نوشته احمدی مربوط نقد سینمایی سینماگرانی چون برسون آندری تارکوفسکی اواخر دهه ۶۰ مجلاتی چون مجله فیلم صاحب امتیازی مدیرمسئولی مسعود مهرابی منتشر شدند کار فلسفی ترجمه اثری والتر بنیامین آغاز بنا گفته ترجمه خوبی نبود بازچاپ جلوگیری سال ۱۳۷۰ کتاب مهمی نوشت عنوان ساختار تاویل 

<div dir="rtl">

به نام ایشمتن به نام خدابیت تنیبا تشنیباتن شاتنبایشمتن 

</div>

## Searching

In [ ]:
from scipy.spatial.distance import cosine

query = 'هخامنشیان ساسانیان هگمتانه'

q_idf = {
    'هخامنشیان': word_idf_rdd.lookup('هخامنشیان')[0] if word_idf_rdd.lookup('هخامنشیان') else math.log((num_articles+1)/1),
    'ساسانیان': word_idf_rdd.lookup('ساسانیان')[0] if word_idf_rdd.lookup('ساسانیان') else math.log((num_articles+1)/1),
    'هگمتانه': word_idf_rdd.lookup('هگمتانه')[0] if word_idf_rdd.lookup('هگمتانه') else math.log((num_articles+1)/1)
}

x = {
    'id': 10,
    'vector': {
    'هخامنشیان': word_idf_rdd.lookup('هخامنشیان')[0] if word_idf_rdd.lookup('هخامنشیان') else 0,
    'ساسانیان': word_idf_rdd.lookup('ساسانیان')[0] if word_idf_rdd.lookup('ساسانیان') else 0,
    'امین': word_idf_rdd.lookup('امین')[0] if word_idf_rdd.lookup('امین') else 0
    }
}

def similarity(x):
  v = x['vector']

  # print('q is: ', q)
  # print('v is: ', v)

  keys = q.keys() | v.keys()
  # print('keys are: ', keys)
  sort = sorted(keys)
  # print('keys after sort: ', sort)

  v_vector = [None]*len(sort)
  q_vector = [None]*len(sort)
  for i, key in enumerate(sort):
    v_vector[i] = v.get(key, 0)
    q_vector[i] = q.get(key, 0)

  # print(v_vector)
  # print(q_vector)

  distance = cosine(q_vector, v_vector)

  # print(distance)

  return (x['id'], distance)

# similarity(x)


# top_10_similar = #TODO: find similar wikipedia articles based on cosine similarity of tf-idf vectors

top_10_similar = articles_cleaned_rdd_with_vector.map(similarity).sortBy(lambda x: x[1])
top_10_similar.take(10)
# top_10_similar = top_10_similar




Py4JJavaError: ignored